# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Mossamedes,100,AO,1611461997,70,-15.1961,12.1522,75.02,5.23
1,1,Ishigaki,75,JP,1611462200,68,24.3448,124.1572,66.20,18.41
2,2,Lorengau,81,PG,1611462200,62,-2.0226,147.2712,85.55,3.02
3,3,Rikitea,0,PF,1611462200,65,-23.1203,-134.9692,79.54,7.52
4,4,Svetlyy Yar,0,RU,1611462200,93,48.4748,44.7810,33.01,8.95
...,...,...,...,...,...,...,...,...,...,...
569,569,Araouane,41,ML,1611462042,23,18.9048,-3.5265,60.42,10.56
570,570,Okhotsk,100,RU,1611462245,81,59.3833,143.3000,-10.77,10.69
571,571,Morogoro,75,TZ,1611462245,94,-6.8167,37.6667,71.60,5.75
572,572,Kenscoff,40,HT,1611462245,88,18.4506,-72.2869,77.00,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [27]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,Rikitea,0,PF,1611462200,65,-23.1203,-134.9692,79.54,7.52
8,8,El Salvador,0,SV,1611462201,88,13.8333,-88.9167,73.40,2.30
58,58,Mar del Plata,0,AR,1611462205,88,-38.0023,-57.5575,75.99,5.75
69,69,São João da Barra,0,BR,1611462022,84,-21.6403,-41.0511,74.08,5.17
217,217,San Ignacio,0,BZ,1611462038,94,17.1561,-89.0714,73.99,6.91
224,224,Wanning,0,CN,1611462218,65,18.8003,110.3967,73.94,5.21
263,263,Pitimbu,0,BR,1611462221,94,-7.4706,-34.8086,75.20,3.44
273,273,Nalgonda,0,IN,1611462222,54,17.0500,79.2667,77.36,8.08
282,282,Makkah al Mukarramah,0,SA,1611462222,47,21.4267,39.8261,70.29,1.57
352,352,Mae Hong Son,0,TH,1611462228,56,19.3003,97.9685,73.40,2.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the results
hotel_df

,City,Country,Lat,Lng,Hotel Name
3,Rikitea,PF,-23.1203,-134.9692,
8,El Salvador,SV,13.8333,-88.9167,
58,Mar del Plata,AR,-38.0023,-57.5575,
69,São João da Barra,BR,-21.6403,-41.0511,
217,San Ignacio,BZ,17.1561,-89.0714,
224,Wanning,CN,18.8003,110.3967,
263,Pitimbu,BR,-7.4706,-34.8086,
273,Nalgonda,IN,17.0500,79.2667,
282,Makkah al Mukarramah,SA,21.4267,39.8261,
352,Mae Hong Son,TH,19.3003,97.9685,


In [32]:
# URL link and parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [33]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 3: Rikitea.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: El Salvador.
Missing field/result... skipping.
------------
Retrieving Results for Index 58: Mar del Plata.
Missing field/result... skipping.
------------
Retrieving Results for Index 69: São João da Barra.
Missing field/result... skipping.
------------
Retrieving Results for Index 217: San Ignacio.
Missing field/result... skipping.
------------
Retrieving Results for Index 224: Wanning.
Missing field/result... skipping.
------------
Retrieving Results for Index 263: Pitimbu.
Missing field/result... skipping.
------------
Retrieving Results for Index 273: Nalgonda.
Missing field/result... skipping.
------------
Retrieving Results for Index 282: Makkah al Mukarramah.
Missing field/result... skipping.
------------
Retrieving Results for Index 352: Mae Hong Son.
Missing field/result... skipping.
------------
Retrieving Results for Index 376: Lātehār.
Missing field/result

In [34]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
3,Rikitea,PF,-23.1203,-134.9692,
8,El Salvador,SV,13.8333,-88.9167,
58,Mar del Plata,AR,-38.0023,-57.5575,
69,São João da Barra,BR,-21.6403,-41.0511,
217,San Ignacio,BZ,17.1561,-89.0714,
224,Wanning,CN,18.8003,110.3967,
263,Pitimbu,BR,-7.4706,-34.8086,
273,Nalgonda,IN,17.0500,79.2667,
282,Makkah al Mukarramah,SA,21.4267,39.8261,
352,Mae Hong Son,TH,19.3003,97.9685,


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))